<a href="https://colab.research.google.com/github/EncodingDavid/tensorflow/blob/master/TensorFlow%E5%A4%9A%E5%B1%82%E6%84%9F%E7%9F%A5%E6%9C%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow


    100% |████████████████████████████████| 73.6MB 143kB/s 
    100% |████████████████████████████████| 2.0MB 276kB/s 
    100% |████████████████████████████████| 102kB 206kB/s 
    100% |████████████████████████████████| 61kB 220kB/s 
    100% |████████████████████████████████| 1.3MB 342kB/s 
    100% |████████████████████████████████| 3.2MB 145kB/s 
    100% |████████████████████████████████| 368kB 194kB/s 
    100% |████████████████████████████████| 61kB 183kB/s 
    100% |████████████████████████████████| 92kB 209kB/s 
  Stored in directory: /Users/david/Library/Caches/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: /Users/david/Library/Caches/pip/wheels/ee/98/38/46cbcc5a93cfea5492d19c38562691ddb23b940176c14f7b48
  Stored in directory: /Users/david/Library/Caches/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built gast absl-py termcolor


In [0]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [0]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
sess = tf.InteractiveSession()

/Users/david/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


## 设置Variable并初始化
因为使用的激活函数是ReLU，所以需要使用正态分布给参数加一点噪声，来打破完全对称并且避免0梯度。

In [0]:
in_units = 784
h1_units = 300
W1 = tf.Variable(tf.truncated_normal([in_units,h1_units],stddev = 0.1))
b1 = tf.Variable(tf.zeros([h1_units]))
W2 = tf.Variable(tf.zeros([h1_units,10]))
b2 = tf.Variable(tf.zeros([10]))

## 定义输入x的placeholder
因为在训练和预测，Dropout的比率keep_prob是不一样的，通常在训练是小于1，而在预测时等于1，所以也把Dropout的比率作为计算图的输入，并定义成一个placeholder

In [0]:
x = tf.placeholder(tf.float32,[None,in_units])
keep_prob = tf.placeholder(tf.float32)

## 定义模型结构
首先需要一个隐层，命名为hidden1，可以通过$tf.nn.relu(tf.matmul(x,W_1)+b_1)$实现一个激活函数为ReLU的隐层，计算公式为$$y=relu(W_1x+b_1)$$
这里的keep_prob参数即为保留数据而不设置为0的比例，在训练时应该是小于1的，用以制造随机性，防止过拟合；在预测时应该等于1，即使用全部特征来预测样本的类别。最后是输出层，也就是第3章介绍的Softmax，这一行代码的功能和之前的是一致的。

In [0]:
hidden1 = tf.nn.relu(tf.matmul(x,W1)+b1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)


In [0]:
y = tf.nn.softmax(tf.matmul(hidden1_drop,W2)+b2)

## 定义损失函数和选择优化器来优化loss 

In [0]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices = [1]))


In [0]:
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)

Instructions for updating:
Use tf.cast instead.


## 训练步骤
- 加入了keep_prob作为计算图的输入
- 在训练时设置0.75，其余25%为0，一般来说，对越复杂越大规模的神经网络，Dropout的效果更显著
- 因为加入了隐层，我们需要更多的训练迭代来优化模型以达到一个比较好的效果
- 所以一共采用来3000个batch，每个batch包含100条样本，一共30万样本


In [0]:
tf.global_variables_initializer().run()
for i in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys, keep_prob: 0.75})

## 对模型准确率进行评估
直接令keep_prob=1，可以达到较好的预测效果

In [0]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images,y_: mnist.test.labels,keep_prob: 1.0}))

0.9796


## 总结
相比之前的Softmax，误差由8%下降到2%，准确率可以说是飞跃性提升。而这个提升仅仅是加入了一个隐层实现，可见多层神经网络的效果有多显著。
当然，我们也加入了Dropout,Adagrad,ReLU等，但决定性的还是隐层，它能对特征进行抽象和转化。

新加入了一个隐层，并使用了Dropout、Adagrad和ReLU，而代码没有增加很多，非常简洁

不过，使用全连接神经网络也是有局限性的，即使我们使用很深的网络，很多的隐藏节点，很大的迭代次数，也很难在MNIST数据集上获得99%以上的准确率，这雨要用到卷积神经网络